# Prerequisites


In [33]:
!pip install glob2

In [34]:
!pip install pytest-shutil

In [35]:
!pip install SimpleITK

In [36]:
from glob import glob
import shutil
import os

import SimpleITK as sitk
import time
import re

# Convertion of .nii files to .dcm files without the help of slicer

In [37]:
def writeSlices(series_tag_values, new_img, i, out_dir):
    image_slice = new_img[:,:,i]
    
    # Set the pixel type explicitly to int16.
    image_slice = sitk.Cast(image_slice, sitk.sitkInt16)
    
    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()

    # Tags shared by the series.
    list(map(lambda tag_value: image_slice.SetMetaData(tag_value[0], tag_value[1]), series_tag_values))

    # Slice specific tags.
    image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
    image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time

    # Setting the type to CT preserves the slice location.
    image_slice.SetMetaData("0008|0060", "CT")  # set the type to CT so the thickness is carried over

    # (0020, 0032) image position patient determines the 3D spacing between slices.
    image_slice.SetMetaData("0020|0032", '\\'.join(map(str,new_img.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
    image_slice.SetMetaData("0020,0013", str(i)) # Instance Number

    # Write to the output directory and add the extension dcm, to force writing in DICOM format.
    writer.SetFileName(os.path.join(out_dir,'slice' + str(i).zfill(4) + '.dcm'))
    writer.Execute(image_slice)

In [38]:
def nifti2dicom_1file(in_dir, out_dir):
    """
    This function is to convert only one nifti file into dicom series
    `nifti_dir`: the path to the one nifti file
    `out_dir`: the path to output
    """

    os.makedirs(out_dir, exist_ok=True)

    new_img = sitk.ReadImage(in_dir) 
    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")

    direction = new_img.GetDirection()
    series_tag_values = [("0008|0031",modification_time), # Series Time
                    ("0008|0021",modification_date), # Series Date
                    ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                    ("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time), # Series Instance UID
                    ("0020|0037", '\\'.join(map(str, (direction[0], direction[3], direction[6],# Image Orientation (Patient)
                                                        direction[1],direction[4],direction[7])))),
                    ("0008|103e", "Created-Pycad")] # Series Description

    # Write slices to output directory
    list(map(lambda i: writeSlices(series_tag_values, new_img, i, out_dir), range(new_img.GetDepth())))

In [39]:
def nifti2dicom_mfiles(nifti_dir, out_dir=''):
    """
    This function is to convert multiple nifti files into dicom files
    `nifti_dir`: You enter the global path to all of the nifti files here.
    `out_dir`: Put the path to where you want to save all the dicoms here.
    PS: Each nifti file's folders will be created automatically, so you do not need to create an empty folder for each patient.
    """

    images = glob(nifti_dir + '/*.nii.gz')

    for image in images:
        o_path = out_dir + '/' + os.path.basename(image)[:-7]
        os.makedirs(o_path, exist_ok=True)

        nifti2dicom_1file(image, o_path)

## give the input directory path

In [40]:
##in_dir = "E:/Thesis/Most_one/LabelsTr"

in_dir = "E:/Thesis/Most_one/ImagesTr"

In [41]:

root = os.path.dirname(in_dir)
print(root)
new_folder_1 = "dicom_initial"
new_folder_2 = "dicom_final"
new_folder_path_1 = os.path.join(root, new_folder_1)
new_folder_path_2 = os.path.join(root, new_folder_2)

try:
    os.mkdir(new_folder_path_1)
    os.mkdir(new_folder_path_2)
except FileExistsError:
    print(f"Folder {new_folder_path_1} & {new_folder_path_2} already exists")
else:
    print(f"Folder {new_folder_path_1} & {new_folder_path_2} created successfully")
    

labels_folder = "labels"
images_folder = "images"
labels_folder_path_1 = os.path.join(new_folder_path_1, labels_folder)
labels_folder_path_2 = os.path.join(new_folder_path_2, labels_folder)

images_folder_path_1 = os.path.join(new_folder_path_1, images_folder)
images_folder_path_2 = os.path.join(new_folder_path_2, images_folder)

## making dicom_initial's labels and images
try:
    os.mkdir(labels_folder_path_1)
    os.mkdir(images_folder_path_1)
except FileExistsError:
    print(f"Folder {labels_folder_path_1} & {images_folder_path_1} already exists")
else:
    print(f"Folder {labels_folder_path_1} created successfully")
    print(f"Folder {images_folder_path_1} created successfully")
    

## making dicom_final's labels and images
try:
    os.mkdir(labels_folder_path_2)
    os.mkdir(images_folder_path_2)
except FileExistsError:
    print(f"Folder {labels_folder_path_2} & {images_folder_path_2} already exists")
else:
    print(f"Folder {labels_folder_path_2} created successfully")
    print(f"Folder {images_folder_path_2} created successfully")



E:/Thesis/Most_one
Folder E:/Thesis/Most_one\dicom_initial & E:/Thesis/Most_one\dicom_final already exists
Folder E:/Thesis/Most_one\dicom_initial\labels & E:/Thesis/Most_one\dicom_initial\images already exists
Folder E:/Thesis/Most_one\dicom_final\labels & E:/Thesis/Most_one\dicom_final\images already exists


### Renaming the copy pasted files


In [42]:
def files_rename(dir_path, keyword):
    
    file_pattern = r'spleen_(\d+)\.nii\.gz'

    for filename in os.listdir(dir_path):
        new_name = keyword + "_" + filename
        try:
            if re.match(file_pattern, filename):
                os.rename(os.path.join(dir_path, filename), os.path.join(dir_path, new_name))
                print(f"success!!! File name: {new_name}")
            else:
                print(f"not renamed due to {new_name}")
        except Exception as e:
            print(f"An error occurred while renaming file '{filename}': {e}")

In [43]:
files_rename(in_dir , "images")

success!!! File name: images_spleen_10.nii.gz
success!!! File name: images_spleen_12.nii.gz
success!!! File name: images_spleen_13.nii.gz
success!!! File name: images_spleen_14.nii.gz
success!!! File name: images_spleen_2.nii.gz
success!!! File name: images_spleen_3.nii.gz
success!!! File name: images_spleen_6.nii.gz
success!!! File name: images_spleen_8.nii.gz
success!!! File name: images_spleen_9.nii.gz


### nifti to dicom (stored in dicom_initial)

In [44]:
nifti2dicom_mfiles(in_dir, images_folder_path_1)

# Divide the .dcm files to multiples folders of same sample size 

In [45]:
print(images_folder_path_1)


E:/Thesis/Most_one\dicom_initial\images


In [46]:
d_initial_images = images_folder_path_1
d_final_images = images_folder_path_2
  
print(os.path.normpath(d_initial_images))
print(d_final_images)

p = glob( d_initial_images +"/*")
print(p)
p = p[0]
print(p)
l = len(glob(p + "/*"))
print(l)

E:\Thesis\Most_one\dicom_initial\images
E:/Thesis/Most_one\dicom_final\images
['E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_10', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_12', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_13', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_14', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_2', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_3', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_6', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_8', 'E:/Thesis/Most_one\\dicom_initial\\images\\images_spleen_9']
E:/Thesis/Most_one\dicom_initial\images\images_spleen_10
55


In [47]:
def dicom_divide(number_of_sice, initial_path , final_path):
    for i in glob(initial_path+"/*"):
        print("Entered-> ",i)

        folder_name = os.path.basename(os.path.normpath(i))
        number_of_folder = int(len(glob(i+"/*"))/number_of_slice)
        print(f"folder_name: {folder_name}")
        print(f"numbers_of_folder: {number_of_folder}")


        for j in range(1,number_of_folder+1):
            new_dir = os.path.join(final_path, folder_name+"_"+str(j))
            print(new_dir)
            
            try:
                os.mkdir(new_dir)
            except OSError as e:
                print(f"Error: {new_dir} : {e.strerror}")
                continue
            
            # get files and copy to the new folder 
            k = 0
            for l in glob(i+"/*"):
                if k == number_of_slice:
                    break

                #shutil.copy(l,new_dir) ---- You have to move or the code has to be changed the code don't copy all the files only first 40 when used copy
                shutil.move(l,new_dir)
                k+=1

In [48]:
#number_of_slice = 40

def number_of_slices( folder_path ):
    
    min_samples = float('inf')
    for i in glob(folder_path+"/*"):
        print("Entered-> ",i)
        
        length = len(glob(i + "/*"))
        
        if(length < min_samples):
            min_samples = length
            print(min_samples)
            
    return min_samples   


In [49]:
number_of_slice = number_of_slices(d_initial_images)
print(number_of_slice)

Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_10
55
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_12
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_13
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_14
54
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_2
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_3
40
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_6
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_8
31
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_9
31


In [50]:
#for images
dicom_divide(number_of_slice,d_initial_images , d_final_images)

Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_10
folder_name: images_spleen_10
numbers_of_folder: 1
E:/Thesis/Most_one\dicom_final\images\images_spleen_10_1
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_12
folder_name: images_spleen_12
numbers_of_folder: 5
E:/Thesis/Most_one\dicom_final\images\images_spleen_12_1
E:/Thesis/Most_one\dicom_final\images\images_spleen_12_2
E:/Thesis/Most_one\dicom_final\images\images_spleen_12_3
E:/Thesis/Most_one\dicom_final\images\images_spleen_12_4
E:/Thesis/Most_one\dicom_final\images\images_spleen_12_5
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_13
folder_name: images_spleen_13
numbers_of_folder: 2
E:/Thesis/Most_one\dicom_final\images\images_spleen_13_1
E:/Thesis/Most_one\dicom_final\images\images_spleen_13_2
Entered->  E:/Thesis/Most_one\dicom_initial\images\images_spleen_14
folder_name: images_spleen_14
numbers_of_folder: 1
E:/Thesis/Most_one\dicom_final\images\images_spleen_14_1
Entered->  

In [51]:
print(len(glob(d_final_images + "/*")))

18


# Convert to nifti files

In [52]:
!pip install dicom2nifti

In [53]:
import dicom2nifti as dn

### create nifti directory

In [54]:
root = os.path.dirname(in_dir)
print(root)
new_folder_3 = "nifti_initial"
new_folder_path_3 = os.path.join(root, new_folder_3)


try:
    os.mkdir(new_folder_path_3)
except FileExistsError:
    print(f"Folder {new_folder_path_3} already exists")
else:
    print(f"Folder {new_folder_path_3} created successfully")

    
labels_folder = "labels"
images_folder = "images"

labels_folder_path_3 = os.path.join(new_folder_path_3, labels_folder)
images_folder_path_3 = os.path.join(new_folder_path_3, images_folder)


## making nifti_initial's labels and images
try:
    os.mkdir(labels_folder_path_3)
    os.mkdir(images_folder_path_3)
except FileExistsError:
    print(f"Folder {labels_folder_path_3} & {images_folder_path_3} already exists")
else:
    print(f"Folder {labels_folder_path_3} created successfully")
    print(f"Folder {images_folder_path_3} created successfully")

E:/Thesis/Most_one
Folder E:/Thesis/Most_one\nifti_initial already exists
Folder E:/Thesis/Most_one\nifti_initial\labels & E:/Thesis/Most_one\nifti_initial\images already exists


In [55]:
dicom_path_images = d_final_images+ "/*"
nifti_path_images = images_folder_path_3

print(dicom_path_images)
print(nifti_path_images)


E:/Thesis/Most_one\dicom_final\images/*
E:/Thesis/Most_one\nifti_initial\images


In [56]:
def dicom_2_nifti(dicom_path , nifti_path):
    folder_list = glob(dicom_path)
    print(folder_list)
    print(nifti_path)
    counter = 0
    
    for i in folder_list:
        folder_name = os.path.basename(os.path.normpath(i))
        dn.dicom_series_to_nifti(i,os.path.join(nifti_path,folder_name+".nii.gz"))
        print(f"completed_{counter}")
        counter = counter + 1


In [57]:
#for images
dicom_2_nifti(dicom_path_images , nifti_path_images)

['E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_10_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_12_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_12_2', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_12_3', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_12_4', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_12_5', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_13_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_13_2', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_14_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_2_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_2_2', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_3_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_6_1', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_6_2', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_6_3', 'E:/Thesis/Most_one\\dicom_final\\images\\images_spleen_6_4'

# Find NULL

In [58]:
!pip install nibabel

In [59]:
import nibabel as nb
import numpy as np

In [60]:
path_of = nifti_path_images + "/*"

In [61]:
list_of_null=[]

for i in glob(path_of):
    nifti = nb.load(i)
    
    frame_data = nifti.get_fdata()
    #print(frame_data)
    
    uniq = np.unique(frame_data)
    print(uniq)
    if len(uniq) == 1:
        list_of_null.append(i)

[-1024. -1023. -1022. ...  3008.  3060.  3072.]
[-1024. -1023. -1022. ...  1436.  1490.  1508.]
[-1024. -1023. -1022. ...  1646.  1652.  1666.]
[-1024. -1023. -1022. ...  1392.  1420.  1432.]
[-1024. -1023. -1022. ...  1326.  1328.  1364.]
[-1024. -1023. -1022. ...  3038.  3058.  3072.]
[-1024. -1023. -1022. ...  1153.  1156.  1189.]
[-1024. -1023. -1022. ...  1116.  1125.  1143.]
[-1024. -1023. -1022. ...  1410.  1734.  1846.]
[-1024. -1023. -1022. ...  3004.  3040.  3071.]
[-1024. -1023. -1022. ...  1462.  1570.  1709.]
[-1024. -1023. -1022. ...  1298.  1322.  1346.]
[-1024. -1023. -1022. ...  1192.  1195.  1197.]
[-1024. -1023. -1022. ...  1185.  1187.  1197.]
[-1024. -1023. -1022. ...  1163.  1171.  1175.]
[-1024. -1023. -1022. ...  1203.  1221.  1249.]
[-1024. -1023. -1022. ...  1140.  1157.  1166.]
[-1024. -1023. -1022. ...  2154.  2626.  2922.]


In [62]:
list_of_null

[]

## Remove NULL files

In [63]:
def remove_null(list_of_null):
    for i in list_of_null:
        if os.path.exists(i):
            os.remove(i)
            print("File deleted successfully.")
        else:
            print("File does not exist.")

In [64]:
 #   remove_null(list_of_null)